In [1]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(""))))

In [2]:
from src.pipeline import Pipeline

[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
pipeline = Pipeline()

In [4]:
pipeline.setup()

In [5]:
pipeline.preprocess()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 613.29it/s]


In [6]:
pipeline.create_all_features()

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 3400.84it/s]


Downloading:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:01<00:00, 79.14it/s]


0 Unknown words replaced with zero vecs

2 Unknown words replaced with zero vecs



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 243/243 [00:00<00:00, 970.66it/s]


In [9]:
pipeline.features

,qID,pID,y,w2v_cosine,w2v_euclidean,w2v_manhattan,tfidf_cosine,tfidf_euclidean,tfidf_manhattan,glove_cosine,...,polarity_doc,subjectivity_query,polarity_query,bm25,doc_nouns,doc_adjectives,doc_verbs,query_nouns,query_adjectives,query_verbs
0,488467,6256112,1,0.999945,77.279285,637.007591,0.466888,1.032581,3.373521,0.642888,...,0.020000,1.0,0.0,-20.174456,14,7,7,2,1,1
1,124599,4924774,1,0.999601,51.363281,423.015234,0.543206,0.955818,3.091751,-0.294182,...,0.000000,0.0,0.0,-12.668757,7,1,2,2,0,0
2,560377,893530,1,0.999901,47.094649,387.909363,0.257147,1.218895,5.017585,0.576715,...,-0.125000,0.0,0.0,-17.635882,13,2,8,0,2,2
3,1015340,3687433,1,0.999943,25.909125,213.380751,0.341013,1.148030,4.683098,0.560266,...,-0.077778,0.3,0.2,-16.037243,14,4,4,4,1,1
5,932835,3292443,1,0.999719,74.538839,613.716448,0.780465,0.662623,3.123014,0.421301,...,0.116667,0.0,0.0,-29.214098,17,3,9,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,427506,5085340,0,0.998925,70.438511,580.073519,0.373732,1.119168,5.287708,0.479431,...,0.370000,0.0,0.0,-19.536848,19,5,5,3,1,1
134,588340,4322103,0,0.999728,105.970543,873.061526,0.100769,1.341068,6.843310,-0.086400,...,-0.044444,0.0,0.0,-8.574309,26,11,5,1,1,2
135,64644,4141830,0,0.999815,73.011524,601.601487,0.369009,1.123380,4.775432,0.499721,...,0.055556,0.0,0.0,-11.243691,10,3,18,1,0,3
136,55778,1342093,0,0.999714,101.530245,836.249584,0.457584,1.041553,4.544277,0.875458,...,0.200000,0.0,0.0,-9.030987,19,3,8,2,1,0


In [13]:
pipeline.evaluate('mlp', 20)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 66.51it/s]


0 Unknown words replaced with zero vecs

0 Unknown words replaced with zero vecs



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:10<00:00, 968.34it/s]
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [14]:
import pandas as pd
from src.utils.utils import load

results = load('data/results/results.pkl')
results

,model,hyperparameters,sampling,pca,MAP,nDCG,accuracy,precision,recall,f1,accuracy@50,precision@50,recall@50,f1@50
0,GaussianNB(),"{'priors': None, 'var_smoothing': 1e-09}",243,0,0.039952,0.210734,0.9561,0.004535,1.0,0.009029,0.01,0.010000,1.0,0.019802
1,GaussianNB(),"{'priors': None, 'var_smoothing': 1e-09}",243,10,1.000000,0.919721,0.9967,0.057143,1.0,0.108108,0.67,0.057143,1.0,0.108108
2,LogisticRegression(),"{'C': 1.0, 'class_weight': None, 'dual': False...",243,10,1.000000,1.000000,1.0000,1.000000,1.0,1.000000,1.00,1.000000,1.0,1.000000
3,MLPClassifier(),"{""activation"": ""relu"", ""alpha"": 0.0001, ""batch...",243,10,1.000000,1.000000,0.9997,0.400000,1.0,0.571429,0.97,0.400000,1.0,0.571429
4,MLPClassifier(),"{""activation"": ""relu"", ""alpha"": 0.0001, ""batch...",243,20,1.000000,1.000000,0.9995,0.285714,1.0,0.444444,0.95,0.285714,1.0,0.444444
